# Simulation of Projectile Motion with Linear Regression and LSTM/RNN

This notebook demonstrates how to simulate projectile motion and use AI algorithms, specifically Linear Regression and LSTM/RNN, to predict the projectile's trajectory.

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
# Constants for the projectile motion
g = 9.81  # Acceleration due to gravity (m/s^2)
theta = 45  # Launch angle (degrees)
v0 = 50  # Initial velocity (m/s)
t_max = 2 * v0 * np.sin(np.radians(theta)) / g  # Total time of flight (s)
t = np.linspace(0, t_max, num=500)  # Time array

# Calculate the trajectory
x = v0 * np.cos(np.radians(theta)) * t
y = v0 * np.sin(np.radians(theta)) * t - 0.5 * g * t**2

# Remove negative y values
valid_indices = y >= 0
x = x[valid_indices]
y = y[valid_indices]
t = t[valid_indices]

# Plot the projectile motion
plt.figure(figsize=(10, 6))
plt.plot(x, y, label='Actual Trajectory')
plt.title('Projectile Motion')
plt.xlabel('Distance (m)')
plt.ylabel('Height (m)')
plt.legend()
plt.show()

In [3]:
# Prepare the data for Linear Regression
data = pd.DataFrame({'x': x, 'y': y})
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

X_train = train_data[['x']]
y_train = train_data['y']
X_test = test_data[['x']]
y_test = test_data['y']

# Linear Regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred_lr = linear_regressor.predict(X_test)

# Plot Linear Regression results
plt.figure(figsize=(10, 6))
plt.plot(x, y, label='Actual Trajectory')
plt.scatter(X_test, y_test, color='red', label='Test Data')
plt.plot(X_test, y_pred_lr, color='green', label='Linear Regression Prediction')
plt.title('Linear Regression for Projectile Motion')
plt.xlabel('Distance (m)')
plt.ylabel('Height (m)')
plt.legend()
plt.show()

# Print Linear Regression MSE
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f'Mean Squared Error (Linear Regression): {mse_lr}')

In [4]:
# Prepare the data for LSTM
sequence_length = 10
features = data[['x', 'y']].values
target = data['y'].values

generator = TimeseriesGenerator(features, target, length=sequence_length, batch_size=1)

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 2)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the LSTM model
model.fit(generator, epochs=10, verbose=1)

# Make predictions using the LSTM model
predictions = []
for i in range(len(data) - sequence_length):
    x_input = features[i:i+sequence_length]
    x_input = x_input.reshape((1, sequence_length, 2))
    yhat = model.predict(x_input, verbose=0)
    predictions.append(yhat[0][0])

# Align predictions for plotting
predictions = np.array(predictions)
predictions = np.concatenate((np.zeros(sequence_length), predictions))

# Plot LSTM results
plt.figure(figsize=(10, 6))
plt.plot(x, y, label='Actual Trajectory')
plt.plot(x, predictions, label='LSTM Prediction', color='orange')
plt.title('LSTM for Projectile Motion Prediction')
plt.xlabel('Distance (m)')
plt.ylabel('Height (m)')
plt.legend()
plt.show()

# Print LSTM MSE
mse_lstm = mean_squared_error(y[sequence_length:], predictions[sequence_length:])
print(f'Mean Squared Error (LSTM): {mse_lstm}')